gmm mit experience driver
customer order
was geordert wird von customer
externe effekte

In [2]:
import importlib
import _1_Preprocessing
importlib.reload(_1_Preprocessing)

<module '_1_Preprocessing' from '/Users/canis/Library/CloudStorage/OneDrive-Persönlich/programme/coding/github/flaschenpost/_1_Preprocessing.py'>

In [3]:
import pandas as pd
from IPython.display import display
import _1_Preprocessing

df_train, df_test = _1_Preprocessing.run_preprocessing()


/Users/canis/Library/CloudStorage/OneDrive-Persönlich/programme/coding/github/flaschenpost/_1_Preprocessing.py:48: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  article_id_dummies = df.groupby('web_order_id')['article_id'].apply(lambda x: pd.Series({f'article_id_{article_id}': 1 for article_id in article_ids_to_encode if article_id in x.values}))


In [4]:
pd.set_option('display.max_columns', None)
display(df_train.head())

,warehouse_id,order_time,has_elevator,floor,is_pre_order,is_business,web_order_id,customer_id,service_time_start,service_time_end,service_time_in_minutes,order_datetime,driver_id,trip_id,article_id,article_weight_in_g,article_id_15043,article_id_18544,article_id_20619,article_id_21243,crate_count_0,crate_count_60,crate_count_45,crate_count_42,crate_count_41,crate_count_43,crate_count_44,crate_count_46,crate_count_47,crate_count_39,crate_count_37,crate_count_35,crate_count_38,crate_count_40,crate_count_50,crate_count_48,crate_count_36,crate_count_33,crate_count_34,crate_count_31,crate_count_52,crate_count_32,crate_count_49,crate_count_28,crate_count_30,crate_count_29,crate_count_27,num_previous_orders_customer,customer_speed
6239674,42,2025-01-20 16:27:34.900000+01:00,False,1.0,False,False,1394710,426781,18,2025-01-20 18:04:28+01:00,1.516666,2025-01-20 16:27:34.900000+01:00,3448,225959,14043.0,28176.0,0.0,0.0,1.0,0.0,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,19,0.0
4852645,9,2024-11-28 19:45:00+01:00,False,1.0,True,False,485887,149765,20,2024-11-28 20:49:33+01:00,7.933333,2024-11-28 19:45:00+01:00,1153,78108,18901.0,60681.0,0.0,0.0,1.0,0.0,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,23,0.0
7207831,24,2024-12-30 09:45:00+01:00,True,5.0,True,True,994985,539457,11,2024-12-30 11:47:35+01:00,44.766666,2024-12-30 09:45:00+01:00,9026,166704,19276.0,192755.0,0.0,0.0,1.0,0.0,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,400,3.0
3056997,26,2024-12-02 08:05:34.210000+01:00,False,0.0,False,True,538571,458989,9,2024-12-02 09:34:30.027000+01:00,5.716666,2024-12-02 08:05:34.210000+01:00,838,86020,20619.0,74555.0,0.0,0.0,1.0,0.0,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,338,0.0
6592214,26,2025-01-15 14:00:00+01:00,True,1.0,True,True,1288491,579824,15,2025-01-15 15:55:30.047000+01:00,18.000000,2025-01-15 14:00:00+01:00,376,210790,22115.0,187574.0,0.0,0.0,1.0,0.0,True,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,143,2.0


In [ ]:
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Define the list of features to keep
features_to_keep = [
    "article_weight_in_g", "is_business", "is_pre_order", "has_elevator", "floor", 
    "num_previous_orders_customer", "customer_speed"
]

# Add all columns that start with 'crate_count_' and 'article_id_'
features_to_keep += [col for col in df_train.columns if col.startswith('crate_count_')]
features_to_keep += [col for col in df_train.columns if col.startswith('article_id_')]

# Create a new dataframe with only the specified features
df_train_filtered = df_train[features_to_keep]
df_test_filtered = df_test[features_to_keep]

# Update X_train and X_test to use the filtered dataframes
X_train = df_train_filtered
X_test = df_test_filtered


# Assuming 'service_time_in_minutes' is the target variable and the rest are features
X_train = df_train.drop(columns=['service_time_in_minutes'])
y_train = df_train['service_time_in_minutes']
X_test = df_test.drop(columns=['service_time_in_minutes'])
y_test = df_test['service_time_in_minutes']

# Initialize and train the Bayesian Ridge Regression model
model = BayesianRidge()
model.fit(X_train, y_train)

# Predict the values for the test set
y_pred = model.predict(X_test)

# Calculate and print the mean squared error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

# Calculate and print the mean absolute error
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')


Mean Squared Error: 115.51959995702732
Mean Absolute Error: 7.360380752971515
